<img src="https://learnopencv.com/wp-content/uploads/2024/05/photo-collage.png" height = 400 width = 600>

**Import Dependencies**

In [6]:
import os
import requests
import zipfile

**Download Files**

In [7]:
# Ensure the directory exists
if not os.path.exists('whisper_examples'):
    os.mkdir('whisper_examples')

def download_file(url, save_name):
    if not os.path.exists(save_name):
        # Handling potential redirection in requests
        with requests.get(url, allow_redirects=True) as r:
            if r.status_code == 200:
                with open(save_name, 'wb') as f:
                    f.write(r.content)
            else:
                print("Failed to download the file, status code:", r.status_code)

def unzip(zip_file=None, target_dir='./whisper_examples'):
    try:
        with zipfile.ZipFile(zip_file, 'r') as z:
            z.extractall(target_dir)
            print("Extracted all to:", target_dir)
    except zipfile.BadZipFile:
        print("Invalid file or error during extraction: Bad Zip File")
    except Exception as e:
        print("An error occurred:", e)

# Correct Dropbox link (Ensure this is the direct download link or properly redirects)
download_url = 'https://www.dropbox.com/scl/fi/gaxpaq6d8aqnbz9mpzlr6/whisper_examples.zip?rlkey=x69vv03tu657bbxbmbe7z322m&st=iabgc5et&dl=1'
save_path = 'whisper_examples/whisper_examples.zip'

download_file(download_url, save_path)
unzip(zip_file=save_path)


Extracted all to: ./whisper_examples


### **[Assembly AI](https://www.assemblyai.com/playground) calling via API**

In [8]:
!pip install -qq assemblyai deepgram-sdk python-dotenv


In [11]:
import time
import assemblyai as aai


# Replace with your API key
aai.settings.api_key = "Assembly AI API KEY HERE"

# URL of the file to transcribe
FILE_URL = "whisper_examples/Old_Farmer.mp3"

config = aai.TranscriptionConfig(
 speaker_labels=True,
 speakers_expected=2
)


transcript = aai.Transcriber().transcribe(FILE_URL, config)


for utterance in transcript.utterances:

   speaker_name = "\033[1m" + ("B" if utterance.speaker == "B" else "A") + "\033[0m"


    # Convert start and end times from milliseconds to seconds
   start_time_seconds = utterance.start / 1000
   end_time_seconds = utterance.end / 1000


   # Convert seconds to MM:SS format
   start_time_mmss = divmod(start_time_seconds, 60)  # divmod returns (minutes, seconds)
   end_time_mmss = divmod(end_time_seconds, 60)


   # Format the MM:SS timestamp
   start_timestamp = f"{int(start_time_mmss[0]):02d}:{int(start_time_mmss[1]):02d}"
   end_timestamp = f"{int(end_time_mmss[0]):02d}:{int(end_time_mmss[1]):02d}"


   print(f"TimeStamp: {start_timestamp}")
   print('\033[1m' + f"SPEAKER  -> {speaker_name}: {utterance.text}")


TimeStamp: 00:01
SPEAKER  -> A: I'm going to introduce you to a rather remarkable man. He's Mister Michael Fitzpatrick from Killenie Maynooth. Now he started to draw the old edge pension in 1927 and seven years ago he got the president's bounty on his hundredth birthday. Now he's from County Clare. He came up from Clare in 1940 to a land commission farm in Maynooth where he lives now you have seen a lot of changes, Mister Fitzpatrick, in farming. What would you say was the biggest change?
TimeStamp: 00:36
SPEAKER  -> B: Well, machinery.
TimeStamp: 00:37
SPEAKER  -> A: And what sort of a machine would you think that made the biggest impression?
TimeStamp: 00:42
SPEAKER  -> B: Well, the reaper and binder is a great one, but by God, the one for cutting up the ground and throwing a crop is a powerful one too.
TimeStamp: 00:51
SPEAKER  -> A: Well, you were. You were saying you're at the time you saw the mowing machine for it made a tremendous impression on you.
TimeStamp: 00:57
SPEAKER  -> 

### **[Deepgram](https://playground.deepgram.com/?endpoint=listen&smart_format=true&diarize=true&language=en&model=nova-2) Calling via API**

In [12]:

import os
from dotenv import load_dotenv

from deepgram import (
    DeepgramClient,
    PrerecordedOptions,
    FileSource,
)

load_dotenv()

# Path to the audio file
AUDIO_FILE = "whisper_examples/Old_Farmer.mp3"

API_KEY = "DeepGram API Key here"


def main():
    try:
        # STEP 1 Create a Deepgram client using the API key
        deepgram = DeepgramClient(API_KEY)

        with open(AUDIO_FILE, "rb") as file:
            buffer_data = file.read()

        payload: FileSource = {
            "buffer": buffer_data,
        }

        #STEP 2: Configure Deepgram options for audio analysis
        options = PrerecordedOptions(
            model="nova-2",
            smart_format=True,
            diarize=True,
            language="en",
        )

        # STEP 3: Call the transcribe_file method with the text payload and options
        response = deepgram.listen.prerecorded.v("1").transcribe_file(payload, options)
       # print(response.to_json["results"]["channels"]["alternatives"]["paragraphs"]["transcript"])

        # STEP 4: Print the response
        print(response.to_json(indent=4))

    except Exception as e:
        print(f"Exception: {e}")


if __name__ == "__main__":
    main()


{
    "metadata": {
        "transaction_key": "deprecated",
        "request_id": "75894eef-92ae-43ad-880f-29a23d4c6c90",
        "sha256": "573d1313e0f6ecbf90688490926c49a7c54dae7cdadd4db7d18d078616f1f4f0",
        "created": "2024-05-14T12:14:09.056Z",
        "duration": 169.896,
        "channels": 1,
        "models": [
            "30089e05-99d1-4376-b32e-c263170674af"
        ],
        "model_info": {
            "30089e05-99d1-4376-b32e-c263170674af": {
                "name": "2-general-nova",
                "version": "2024-01-09.29447",
                "arch": "nova-2"
            }
        }
    },
    "results": {
        "channels": [
            {
                "alternatives": [
                    {
                        "transcript": "I'm going to introduce you to a rather remarkable man. He's mister Michael Fitzpatrick from Kilenny, Maynooth. Now he started to draw the old age pension in 1927, and 7 years ago, he got the president's bounty on his 100th birthday

In [ ]:
!curl \
  --request POST \
  --url 'https://api.deepgram.com/v1/listen?diarize=true&punctuate=true&utterances=true' \
  --header 'Authorization: Token DG_API_KEY' \
  --header 'content-type: whisper_examples/Old_Farmer.mp3' \
  --data-binary @whisper_examples/Old_Farmer.mp3

 #Replace you DGram API Key here

{"metadata":{"transaction_key":"deprecated","request_id":"23fa9270-a5af-48aa-a26a-a3a8791ae657","sha256":"573d1313e0f6ecbf90688490926c49a7c54dae7cdadd4db7d18d078616f1f4f0","created":"2024-05-13T16:29:46.997Z","duration":169.896,"channels":1,"models":["1ed36bac-f71c-4f3f-a31f-02fd6525c489"],"model_info":{"1ed36bac-f71c-4f3f-a31f-02fd6525c489":{"name":"general","version":"2024-01-26.8851","arch":"base"}}},"results":{"channels":[{"alternatives":[{"transcript":"I'm going to introduce you to a rather remarkable man. He's mister Mike Fitzpatrick from Kill Man. Now, he started to draw the old lens pension in nineteen twenty seven, and seven years ago, he got the president's bounty, on his hundred birthday. Now he's from County Claire. He came up from Claire in nineteen forty to a land commission farm in Man where he lives now. You have seen a lot of changes, mister Fitzpatrick in farming What would just say was the biggest change? When machinery? And what sort of a machine when you think that